# Goodcab Cab Services

Introduction
- Goodcab is a cab service company established in 2022, it has gained strong foothold in indian market with its focus in tier 2- cities.
- Goodcab operates in ten tier-2 cities across india and has set ambitious performance target for 2024 to drive growth and improve passenger satisfaction.

### Import libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option("display.max_columns", None)
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

### Load the data

In [2]:
#load the city data
city = pd.read_csv("dim_city.csv")
city.head(2)

,city_id,city_name
0,RJ01,Jaipur
1,UP01,Lucknow


In [3]:
#load the date data
calendar = pd.read_csv("dim_date.csv")
calendar.head(3)

,date,start_of_month,month_name,day_type
0,2024-01-01,2024-01-01,January,Weekday
1,2024-01-02,2024-01-01,January,Weekday
2,2024-01-03,2024-01-01,January,Weekday


In [4]:
#load the repeat_trip distribution data
repeat_trip = pd.read_csv('dim_repeat_trip_distribution.csv')
repeat_trip.head(3)

,month,city_id,trip_count,repeat_passenger_count
0,2024-01-01,AP01,10-Trips,7
1,2024-01-01,AP01,2-Trips,352
2,2024-01-01,AP01,3-Trips,158


In [5]:
#load passenger_summary data
passenger_summary = pd.read_csv('fact_passenger_summary.csv')
passenger_summary.head(3)

,month,city_id,new_passengers,repeat_passengers,total_passengers
0,2024-01-01,AP01,2513,650,3163
1,2024-01-01,CH01,3920,720,4640
2,2024-01-01,GJ01,2432,1184,3616


In [6]:
#load the trip data
trips = pd.read_csv("fact_trips.csv")
trips.head(3)

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5
1,TRPVAD240129a3b6dba8,2024-01-29,GJ02,repeated,7,74,5,5
2,TRPCOI240107a42430fb,2024-01-07,TN01,repeated,11,155,8,8


In [7]:
#load the month_target_new_passenger data
month_target_new_passenger = pd.read_csv('monthly_target_new_passengers.csv')
month_target_new_passenger.head(3)

,month,city_id,target_new_passengers
0,2024-05-01,GJ01,1500
1,2024-05-01,GJ02,1500
2,2024-03-01,GJ01,2000


In [8]:
#load the monthly_target_trips data
monthly_target_trips = pd.read_csv('monthly_target_trips.csv')
monthly_target_trips.head(3)

,month,city_id,total_target_trips
0,2024-03-01,MP01,7000
1,2024-05-01,KA01,2500
2,2024-04-01,UP01,11000


In [9]:
#load the ratings data
ratings = pd.read_csv('city_target_passenger_rating.csv')
ratings.head(3)

,city_id,target_avg_passenger_rating
0,CH01,8.00
1,UP01,7.25
2,AP01,8.50


## Data Wrangling

### City Table

In [10]:
#check shape
city.shape

(10, 2)

In [11]:
#check missing values
city.isna().sum()

city_id      0
city_name    0
dtype: int64

In [12]:
#check for duplicates
city.duplicated().sum()

0

In [13]:
#check for unique cities
city['city_name'].unique()

array(['Jaipur', 'Lucknow', 'Surat', 'Kochi', 'Indore', 'Chandigarh',
       'Vadodara', 'Visakhapatnam', 'Coimbatore', 'Mysore'], dtype=object)

### Trips table

In [14]:
#check for shape
trips.shape

(425903, 8)

In [15]:
#check for missing values
trips.isna().sum()

trip_id                   0
date                      0
city_id                   0
passenger_type            0
distance_travelled(km)    0
fare_amount               0
passenger_rating          0
driver_rating             0
dtype: int64

- There are no missing data in the fact trip table

In [16]:
#check for duplicates
trips.duplicated().sum()

0

- There are no duplicates in the trips table

### Passenger Summary Table

In [17]:
#check the shape
passenger_summary.shape

(60, 5)

In [18]:
#check missing data
passenger_summary.isna().sum()

month                0
city_id              0
new_passengers       0
repeat_passengers    0
total_passengers     0
dtype: int64

In [19]:
#Check for duplicates
passenger_summary.duplicated().sum()

0

In [20]:
#check datatypes
passenger_summary.dtypes

month                object
city_id              object
new_passengers        int64
repeat_passengers     int64
total_passengers      int64
dtype: object

## Data Cleaning & Feature Engineering

In [21]:
#change date to date datatypes
calendar['date'] = calendar['date'].astype('datetime64')

#### REPEAT TRIP TABLE

In [22]:
#change the date for repeat_trip table
repeat_trip['month'] = repeat_trip['month'].astype('datetime64')

In [23]:
#extract month_index and month_name for the repeat_trip
repeat_trip['month_index'] = repeat_trip['month'].dt.month
repeat_trip['month_name'] = repeat_trip['month'].dt.month_name()

In [24]:
#merge the city name to the repeat_trip table
repeat_trip = repeat_trip.merge(city, on='city_id')

In [25]:
#display the head
repeat_trip.head()

,month,city_id,trip_count,repeat_passenger_count,month_index,month_name,city_name
0,2024-01-01,AP01,10-Trips,7,1,January,Visakhapatnam
1,2024-01-01,AP01,2-Trips,352,1,January,Visakhapatnam
2,2024-01-01,AP01,3-Trips,158,1,January,Visakhapatnam
3,2024-01-01,AP01,4-Trips,53,1,January,Visakhapatnam
4,2024-01-01,AP01,5-Trips,38,1,January,Visakhapatnam


#### PASSENGER SUMMARY TABLE

In [26]:
#change datatype
passenger_summary['month'] = passenger_summary['month'].astype('datetime64')

In [27]:
#extract month_index and month_name
passenger_summary['month_index'] = passenger_summary['month'].dt.month
passenger_summary['month_name'] = passenger_summary['month'].dt.month_name()

In [28]:
#merge city name
passenger_summary = passenger_summary.merge(city, on='city_id')
passenger_summary.head()

,month,city_id,new_passengers,repeat_passengers,total_passengers,month_index,month_name,city_name
0,2024-01-01,AP01,2513,650,3163,1,January,Visakhapatnam
1,2024-02-01,AP01,2380,790,3170,2,February,Visakhapatnam
2,2024-03-01,AP01,2170,923,3093,3,March,Visakhapatnam
3,2024-04-01,AP01,1845,992,2837,4,April,Visakhapatnam
4,2024-05-01,AP01,1939,951,2890,5,May,Visakhapatnam


#### TRIP TABLE

In [29]:
#change datatype
trips['date'] = trips['date'].astype('datetime64')

In [30]:
#extract month_index and month_name
trips['month_index'] = trips['date'].dt.month
trips['month_name'] = trips['date'].dt.month_name()

In [31]:
#merge city name
trips = trips.merge(city, on='city_id')
trips.head()

,trip_id,date,city_id,passenger_type,distance_travelled(km),fare_amount,passenger_rating,driver_rating,month_index,month_name,city_name
0,TRPLUC240113d55de2fb,2024-01-13,UP01,repeated,11,158,5,5,1,January,Lucknow
1,TRPLUC240327a61cfe66,2024-03-27,UP01,new,5,71,7,8,3,March,Lucknow
2,TRPLUC240322c73f8439,2024-03-22,UP01,repeated,15,161,6,7,3,March,Lucknow
3,TRPLUC240420c114cb02,2024-04-20,UP01,repeated,13,182,7,5,4,April,Lucknow
4,TRPLUC2405094011dfa0,2024-05-09,UP01,repeated,18,188,6,5,5,May,Lucknow


#### MONTH TARGET NEW PASSENGER

In [32]:
#change datatype
month_target_new_passenger['month'] = month_target_new_passenger['month'].astype('datetime64')

In [33]:
#extract month_index and month_name
month_target_new_passenger['month_index'] = month_target_new_passenger['month'].dt.month
month_target_new_passenger['month_name'] = month_target_new_passenger['month'].dt.month_name()

In [34]:
#merge city name
month_target_new_passenger = month_target_new_passenger.merge(city, on='city_id')
month_target_new_passenger.head()

,month,city_id,target_new_passengers,month_index,month_name,city_name
0,2024-05-01,GJ01,1500,5,May,Surat
1,2024-03-01,GJ01,2000,3,March,Surat
2,2024-02-01,GJ01,2000,2,February,Surat
3,2024-04-01,GJ01,1500,4,April,Surat
4,2024-01-01,GJ01,2000,1,January,Surat


In [35]:
month_target_new_passenger.shape

(60, 6)

#### MONTHLY TARGET TRIPS

In [36]:
#change datatype
monthly_target_trips['month'] = monthly_target_trips['month'].astype('datetime64')

In [37]:
#extract month_index and month_name
monthly_target_trips['month_index'] = monthly_target_trips['month'].dt.month
monthly_target_trips['month_name'] = monthly_target_trips['month'].dt.month_name()

In [38]:
#merge city name
monthly_target_trips = monthly_target_trips.merge(city, on='city_id')
monthly_target_trips.head()

,month,city_id,total_target_trips,month_index,month_name,city_name
0,2024-03-01,MP01,7000,3,March,Indore
1,2024-04-01,MP01,7500,4,April,Indore
2,2024-02-01,MP01,7000,2,February,Indore
3,2024-05-01,MP01,7500,5,May,Indore
4,2024-06-01,MP01,7500,6,June,Indore


In [39]:
monthly_target_trips.shape

(60, 6)

#### RATINGS TABLE

In [40]:
#merge city name
ratings = ratings.merge(city, on='city_id')
ratings.head()

,city_id,target_avg_passenger_rating,city_name
0,CH01,8.00,Chandigarh
1,UP01,7.25,Lucknow
2,AP01,8.50,Visakhapatnam
3,MP01,8.00,Indore
4,RJ01,8.25,Jaipur


In [41]:
#check shape
ratings.shape

(10, 3)

## KPI's (Metrics)

### Total trips

In [42]:
#total trips
total_trips = trips['trip_id'].count()
total_trips

425903

### Total Revenue

In [43]:
#Total revenue
total_revenue = trips['fare_amount'].sum()
total_revenue

108188091

### Total Distance Travelled

In [44]:
# total distance
total_distance = trips['distance_travelled(km)'].sum()
print(f"{total_distance:,}")

8,146,320


### Average fare per trip

In [45]:
#average fare per trip
avg_fare_per_trip = sum(trips['fare_amount']) / trips['trip_id'].count()
print(f"{avg_fare_per_trip:,}")

254.0204952770936


### Average fare per distance travelled

In [46]:
#average fare per distance
avg_fare_per_distance = sum(trips['fare_amount']) / sum(trips['distance_travelled(km)'])
avg_fare_per_distance

13.280609035736381

### Average trip distance

In [47]:
# Average trip distance
avg_trip_distance = trips['distance_travelled(km)'].mean()
avg_trip_distance

19.127172149527006

### Maximum and Minimum trip distance

In [48]:
max_distance = trips['distance_travelled(km)'].max()
max_distance

45

In [49]:
min_distance = trips['distance_travelled(km)'].min()
min_distance

5

### Trips Types

In [50]:
# total new trips and repeated trip
trips['passenger_type'].value_counts(normalize=True)

repeated    0.584417
new         0.415583
Name: passenger_type, dtype: float64

- The number of trips that consists of repeats is 248,905
- While, the number of trips that consists of new passengers are 176,998

### Total Passengers

In [51]:
# total passengers
total_passengers = passenger_summary['total_passengers'].sum()
print(f"{total_passengers:,}")

238,309


In [52]:
#total new passengers
new_passengers = passenger_summary['new_passengers'].sum()
print(f"{new_passengers:,}")

176,998


In [53]:
#total repeat passengers
repeat_passengers = passenger_summary['repeat_passengers'].sum()
print(f"{repeat_passengers:,}")

61,311


### Repeat Passenger Rate

- is a metric that measures how many passengers chooses to ride with goodcabs again
- it is the percentage of passengers who make more than one ride with goodcabs during a given period
- it is calculated by dividing the number of repeat passengers by the total number of passengers then multiply by 100
- it show ongoing engagement and loyalty, which can boost a customers lifetime value
- it helps to show certain behavioural trends if this metric is high at certain month or week or time

### Repeat Passenger Rate by month

In [54]:
#grouping by month index and month to summarize the number of repeat and total passengers
rpr_month = passenger_summary.groupby(['month_index', 'month_name'], as_index=False).agg(
    repeat_passengers = ('repeat_passengers', 'sum'),
    total_passengers = ('total_passengers', 'sum')
)
rpr_month

,month_index,month_name,repeat_passengers,total_passengers
0,1,January,8343,44672
1,2,February,9523,45724
2,3,March,10584,41398
3,4,April,11013,37633
4,5,May,12167,36349
5,6,June,9681,32533


In [55]:
#calculating the repeat passenger rate
rpr_month['%_rpr'] = rpr_month['repeat_passengers'] * 100 / rpr_month['total_passengers']

In [56]:
#displaying the result
rpr_month

,month_index,month_name,repeat_passengers,total_passengers,%_rpr
0,1,January,8343,44672,18.676128
1,2,February,9523,45724,20.827137
2,3,March,10584,41398,25.566452
3,4,April,11013,37633,29.264210
4,5,May,12167,36349,33.472723
5,6,June,9681,32533,29.757477


- The result show a steady increase in repeat passsenger rate from january and peaked to about 33% in may, then it dropped to 29% in june
- The month of May is the month of festivities in india, and this may have resulted in a very high repeat passenger rate

### Repeat Passenger Rate by City

In [57]:
#getting the total passenger and repeat by city
rpr_city = passenger_summary.groupby(['city_name'], as_index=False).agg(
    repeat_passengers = ('repeat_passengers', 'sum'),
    total_passengers = ('total_passengers', 'sum')
)

rpr_city

,city_name,repeat_passengers,total_passengers
0,Chandigarh,5070,23978
1,Coimbatore,2551,11065
2,Indore,7216,22079
3,Jaipur,9682,55538
4,Kochi,7626,34042
5,Lucknow,9597,25857
6,Mysore,1477,13158
7,Surat,8638,20264
8,Vadodara,4346,14473
9,Visakhapatnam,5108,17855


In [58]:
#calculating % of repeat passenger
rpr_city['%_rpr'] = rpr_city['repeat_passengers'] * 100 / rpr_city['total_passengers']

In [59]:
#displaying the result
rpr_city

,city_name,repeat_passengers,total_passengers,%_rpr
0,Chandigarh,5070,23978,21.144382
1,Coimbatore,2551,11065,23.054677
2,Indore,7216,22079,32.682640
3,Jaipur,9682,55538,17.433109
4,Kochi,7626,34042,22.401739
5,Lucknow,9597,25857,37.115675
6,Mysore,1477,13158,11.225110
7,Surat,8638,20264,42.627319
8,Vadodara,4346,14473,30.028329
9,Visakhapatnam,5108,17855,28.608233


- Surat, Lucknow and, Indore are the top three cities with high repeat passenger rates

## New vs repeat Trip ratio

- The new and repeat customer ratio is a metric that measures the number of repeat customers for every new customer acquired over a specific period of time
- it is calculated as = new passengers acquired / repeat passengers
- it can be calculated monthly, or across the six-month period
- a high ratio of new passengers compared to repeat indicates that there are issues in retaining those passengers

### New vs Repeat Trip ratio by Month

In [60]:
#filter trip by passenger type for new and group by month index name and passenger type
new_trips_month = trips.query("passenger_type == 'new'").groupby(['month_index', 'month_name', 'passenger_type'], as_index=False).agg(
    new_trips=('trip_id', 'count')
).drop('passenger_type', axis=1)

new_trips_month

,month_index,month_name,new_trips
0,1,January,36329
1,2,February,36201
2,3,March,30814
3,4,April,26620
4,5,May,24182
5,6,June,22852


In [61]:
#filter trip by passenger type for repeated and group by month index name and passenger type
repeat_trips_month = trips.query("passenger_type == 'repeated'").groupby(['month_index', 'month_name', 'passenger_type'], as_index=False).agg(
    repeated_trips=('trip_id', 'count')
).drop('passenger_type', axis=1)

repeat_trips_month

,month_index,month_name,repeated_trips
0,1,January,34133
1,2,February,39178
2,3,March,42865
3,4,April,44715
4,5,May,48361
5,6,June,39653


In [62]:
#merge the new_trips_month and repeat_trips_month
new_repeat_trip_ratio =new_trips_month.merge(repeat_trips_month, on=['month_index', 'month_name'])
new_repeat_trip_ratio

,month_index,month_name,new_trips,repeated_trips
0,1,January,36329,34133
1,2,February,36201,39178
2,3,March,30814,42865
3,4,April,26620,44715
4,5,May,24182,48361
5,6,June,22852,39653


In [63]:
#calculate the new_repeat_trip_ratio
new_repeat_trip_ratio['new_repeat_trip_ratio'] = new_repeat_trip_ratio['new_trips'] / new_repeat_trip_ratio['repeated_trips']
new_repeat_trip_ratio

,month_index,month_name,new_trips,repeated_trips,new_repeat_trip_ratio
0,1,January,36329,34133,1.064337
1,2,February,36201,39178,0.924013
2,3,March,30814,42865,0.718862
3,4,April,26620,44715,0.595326
4,5,May,24182,48361,0.500031
5,6,June,22852,39653,0.576299


- There is a decrease in new_repeat_trip_ratio from january to may to about 0.5
- This means that in the month of May, for every one(1) new trip achieved there are two(2) repeated trip gained 

### New vs Repeat Trip ratio by City

In [64]:
#calculating the new trips based on cities
new_city_trip = trips.query("passenger_type=='new'").groupby(['city_name', 'passenger_type'], as_index=False).agg(
    new_trips = ('trip_id', 'count')
).drop('passenger_type', axis=1)

new_city_trip

,city_name,new_trips
0,Chandigarh,18908
1,Coimbatore,8514
2,Indore,14863
3,Jaipur,45856
4,Kochi,26416
5,Lucknow,16260
6,Mysore,11681
7,Surat,11626
8,Vadodara,10127
9,Visakhapatnam,12747


In [65]:
#calculating the repeat trips based on cities
repeat_city_trip = trips.query("passenger_type=='repeated'").groupby(['city_name', 'passenger_type'], as_index=False).agg(
    repeat_trips = ('trip_id', 'count')
).drop('passenger_type', axis=1)

repeat_city_trip

,city_name,repeat_trips
0,Chandigarh,20073
1,Coimbatore,12590
2,Indore,27593
3,Jaipur,31032
4,Kochi,24286
5,Lucknow,48039
6,Mysore,4557
7,Surat,43217
8,Vadodara,21899
9,Visakhapatnam,15619


In [66]:
#merge the two dataframe
city_new_repeat_trip_ratio = new_city_trip.merge(repeat_city_trip, on='city_name')
city_new_repeat_trip_ratio

,city_name,new_trips,repeat_trips
0,Chandigarh,18908,20073
1,Coimbatore,8514,12590
2,Indore,14863,27593
3,Jaipur,45856,31032
4,Kochi,26416,24286
5,Lucknow,16260,48039
6,Mysore,11681,4557
7,Surat,11626,43217
8,Vadodara,10127,21899
9,Visakhapatnam,12747,15619


In [67]:
#calculating the new_repeat_trip_ratio
city_new_repeat_trip_ratio['new_repeat_trip_ratio'] = city_new_repeat_trip_ratio['new_trips']/city_new_repeat_trip_ratio['repeat_trips']
city_new_repeat_trip_ratio.sort_values('new_repeat_trip_ratio', ascending=False)

,city_name,new_trips,repeat_trips,new_repeat_trip_ratio
6,Mysore,11681,4557,2.563309
3,Jaipur,45856,31032,1.477700
4,Kochi,26416,24286,1.087705
0,Chandigarh,18908,20073,0.941962
9,Visakhapatnam,12747,15619,0.816121
1,Coimbatore,8514,12590,0.676251
2,Indore,14863,27593,0.538651
8,Vadodara,10127,21899,0.462441
5,Lucknow,16260,48039,0.338475
7,Surat,11626,43217,0.269015


- Mysore had the highest new_repeat_trip ratio, meaning that for mysore, for every one repeated trip, there was two new trip gained

## New vs Repeat Passenger ratio

#### New-vs-Repeat Passenger by Month

In [68]:
#getting the new_repeat_passenger_month
new_repeat_passenger_month = passenger_summary.groupby(['month_index', 'month_name'], as_index=False).agg(
    new_passengers = ('new_passengers', 'sum'),
    repeat_passengers = ('repeat_passengers', 'sum')
)

new_repeat_passenger_month

,month_index,month_name,new_passengers,repeat_passengers
0,1,January,36329,8343
1,2,February,36201,9523
2,3,March,30814,10584
3,4,April,26620,11013
4,5,May,24182,12167
5,6,June,22852,9681


In [69]:
#calculate the new_repeat_passenger_ratio
new_repeat_passenger_month['new_repeat_passenger_ratio'] = new_repeat_passenger_month['new_passengers'] / new_repeat_passenger_month['repeat_passengers']

In [70]:
#display the result
new_repeat_passenger_month

,month_index,month_name,new_passengers,repeat_passengers,new_repeat_passenger_ratio
0,1,January,36329,8343,4.354429
1,2,February,36201,9523,3.801428
2,3,March,30814,10584,2.911376
3,4,April,26620,11013,2.417143
4,5,May,24182,12167,1.987507
5,6,June,22852,9681,2.360500


- for every one repeat passenger, 4.3 new passenges are gained in the month of january. 
- This means that the rate of gaining new passenger was 4 times the number of repeat in the month of january
- This begins to drop each month 

#### New-vs-Repeat Passenger by City

In [71]:
#grouping by city to get the new passengers and repeat passengers
new_repeat_passenger_city = passenger_summary.groupby(['city_name'], as_index=False).agg(
    new_passengers = ('new_passengers', 'sum'),
    repeat_passengers = ('repeat_passengers', 'sum')
)

new_repeat_passenger_city

,city_name,new_passengers,repeat_passengers
0,Chandigarh,18908,5070
1,Coimbatore,8514,2551
2,Indore,14863,7216
3,Jaipur,45856,9682
4,Kochi,26416,7626
5,Lucknow,16260,9597
6,Mysore,11681,1477
7,Surat,11626,8638
8,Vadodara,10127,4346
9,Visakhapatnam,12747,5108


In [72]:
#calculating the new_repeat_passenger_ratio
new_repeat_passenger_city['new_repeat_passenger_ratio'] = new_repeat_passenger_city['new_passengers'] / new_repeat_passenger_city['repeat_passengers']

In [73]:
#display the results
new_repeat_passenger_city.sort_values('new_repeat_passenger_ratio')

,city_name,new_passengers,repeat_passengers,new_repeat_passenger_ratio
7,Surat,11626,8638,1.345913
5,Lucknow,16260,9597,1.694279
2,Indore,14863,7216,2.059728
8,Vadodara,10127,4346,2.330189
9,Visakhapatnam,12747,5108,2.495497
1,Coimbatore,8514,2551,3.337515
4,Kochi,26416,7626,3.463939
0,Chandigarh,18908,5070,3.729389
3,Jaipur,45856,9682,4.736212
6,Mysore,11681,1477,7.908599


- A high ratio of new-to-repeat passengers indicate that the goodcabs in those city are gaining new passengers quickly and are having trouble retaining these passengers based on the low repeat-to-new passenger ratio
- In mysore, the new_repeat_ratio is very high, indicating it gain new passenger fast but repeat_new_ratio is the lowest, which means that the city have trouble retaining this passengers

### Revenue Growth Rate

- is a financial metric that measures a company revenue increase or decrease over a period of time
- it is calculated by comparing the current periods revenue to the previous period revenue
- (current revenue - previous revenue) / previous revenue
- it can be calculated on monthly, quarterly or yearly basis

In [74]:
#get the current month revenue
revenue_month = trips.groupby(['month_index', 'month_name'], as_index=False).agg(
    current_revenue = ('fare_amount', 'sum')
)

revenue_month

,month_index,month_name,current_revenue
0,1,January,18454142
1,2,February,19859356
2,3,March,18836382
3,4,April,17695759
4,5,May,17985168
5,6,June,15357284


In [75]:
#creating a previous revenue 

#create a variable called prev_revenue
prev_revenue = [np.nan]

for x in range(0, len(revenue_month)-1):
    y = revenue_month['current_revenue'].iloc[x]
    prev_revenue.append(y)


#create a last_month revenue
revenue_month['last_month_revenue'] = prev_revenue

In [76]:
#display the results
revenue_month

,month_index,month_name,current_revenue,last_month_revenue
0,1,January,18454142,NaN
1,2,February,19859356,18454142.0
2,3,March,18836382,19859356.0
3,4,April,17695759,18836382.0
4,5,May,17985168,17695759.0
5,6,June,15357284,17985168.0


In [77]:
#calculate the revenue growth rate
revenue_month['revenue_growth_rate'] = (revenue_month['current_revenue'] - revenue_month['last_month_revenue']) *100 / revenue_month['last_month_revenue']
revenue_month

,month_index,month_name,current_revenue,last_month_revenue,revenue_growth_rate
0,1,January,18454142,NaN,NaN
1,2,February,19859356,18454142.0,7.614627
2,3,March,18836382,19859356.0,-5.151094
3,4,April,17695759,18836382.0,-6.055425
4,5,May,17985168,17695759.0,1.635471
5,6,June,15357284,17985168.0,-14.611395


- There was a 7.6% increase in revenue in february, this drop by 5% im march and 6% in april
- The month of may withness a slight increase by 1.6% and by june, there was a drastical drop in revenue by 14%

### Revenue by City

In [78]:
#creating revenue by city
revenue_city = trips.groupby(['city_name'], as_index=False).agg(
    total_revenue = ('fare_amount', 'sum')
)

revenue_city['%_revenue_contribution'] = round((revenue_city['total_revenue'] * 100) / trips['fare_amount'].sum(), 1)

In [79]:
#display the result
revenue_city = revenue_city.sort_values('%_revenue_contribution', ascending=False)
revenue_city

,city_name,total_revenue,%_revenue_contribution
3,Jaipur,37207497,34.4
4,Kochi,16997596,15.7
0,Chandigarh,11058401,10.2
5,Lucknow,9463551,8.7
9,Visakhapatnam,8018282,7.4
2,Indore,7635228,7.1
7,Surat,6431599,5.9
6,Mysore,4054745,3.7
8,Vadodara,3797200,3.5
1,Coimbatore,3523992,3.3


## Target Achievement Rate

- is a metric that measures how well the goodcabs met their targets
- it is calculated by dividing the actual by the target and multiply by 100

### City-Level Actual Trips vs Target Trip Achievement Rate

In [80]:
# total actual trips by cities
city_level_actual_trips =trips.groupby(['city_name'], as_index=False).agg(
    total_actual_trips = ('trip_id', 'count')
).sort_values('city_name', ascending=True)

city_level_actual_trips

,city_name,total_actual_trips
0,Chandigarh,38981
1,Coimbatore,21104
2,Indore,42456
3,Jaipur,76888
4,Kochi,50702
5,Lucknow,64299
6,Mysore,16238
7,Surat,54843
8,Vadodara,32026
9,Visakhapatnam,28366


In [81]:
#city_level target trips
city_level_target_trips = monthly_target_trips.groupby(['city_name'], as_index=False).agg(
    total_target_trips = ('total_target_trips', 'sum')
).sort_values('city_name', ascending=True)

city_level_target_trips

,city_name,total_target_trips
0,Chandigarh,39000
1,Coimbatore,21000
2,Indore,43500
3,Jaipur,67500
4,Kochi,49500
5,Lucknow,72000
6,Mysore,13500
7,Surat,57000
8,Vadodara,37500
9,Visakhapatnam,28500


In [82]:
# merge the city_level actual trips with the city_level_target_trips
city_actual_target_trips = city_level_actual_trips.merge(city_level_target_trips, on='city_name')
city_actual_target_trips

,city_name,total_actual_trips,total_target_trips
0,Chandigarh,38981,39000
1,Coimbatore,21104,21000
2,Indore,42456,43500
3,Jaipur,76888,67500
4,Kochi,50702,49500
5,Lucknow,64299,72000
6,Mysore,16238,13500
7,Surat,54843,57000
8,Vadodara,32026,37500
9,Visakhapatnam,28366,28500


In [83]:
#calculate the target achievement rate
city_actual_target_trips['target_achievement_rate'] = city_actual_target_trips['total_actual_trips'] * 100 / city_actual_target_trips['total_target_trips']
city_actual_target_trips.sort_values('target_achievement_rate', ascending=False)

,city_name,total_actual_trips,total_target_trips,target_achievement_rate
6,Mysore,16238,13500,120.281481
3,Jaipur,76888,67500,113.908148
4,Kochi,50702,49500,102.428283
1,Coimbatore,21104,21000,100.495238
0,Chandigarh,38981,39000,99.951282
9,Visakhapatnam,28366,28500,99.529825
2,Indore,42456,43500,97.600000
7,Surat,54843,57000,96.215789
5,Lucknow,64299,72000,89.304167
8,Vadodara,32026,37500,85.402667


### Actual vs Target Trip Achievement by Month 

In [84]:
#calculate the total_actual_trips by month

monthly_actual_trips = trips.groupby(['month_index', 'month_name'], as_index=False).agg(
    total_actual_trips = ('trip_id', 'count')
)

monthly_actual_trips

,month_index,month_name,total_actual_trips
0,1,January,70462
1,2,February,75379
2,3,March,73679
3,4,April,71335
4,5,May,72543
5,6,June,62505


In [85]:
#get the total target trips by month

target_month_trips = monthly_target_trips.groupby(['month_index', 'month_name'], as_index=False).agg(
    total_target_trips = ('total_target_trips', 'sum')
)

target_month_trips

,month_index,month_name,total_target_trips
0,1,January,72500
1,2,February,72500
2,3,March,72500
3,4,April,70500
4,5,May,70500
5,6,June,70500


In [86]:
#merge the month_actual_trips with the target_month_trips

month_actual_target_trips = monthly_actual_trips.merge(target_month_trips, on=['month_index', 'month_name'])
month_actual_target_trips

,month_index,month_name,total_actual_trips,total_target_trips
0,1,January,70462,72500
1,2,February,75379,72500
2,3,March,73679,72500
3,4,April,71335,70500
4,5,May,72543,70500
5,6,June,62505,70500


In [87]:
#calculate the target achievement rate

month_actual_target_trips['target_achievement_rate'] = month_actual_target_trips['total_actual_trips'] * 100 / month_actual_target_trips['total_target_trips']
month_actual_target_trips

,month_index,month_name,total_actual_trips,total_target_trips,target_achievement_rate
0,1,January,70462,72500,97.188966
1,2,February,75379,72500,103.971034
2,3,March,73679,72500,101.626207
3,4,April,71335,70500,101.184397
4,5,May,72543,70500,102.897872
5,6,June,62505,70500,88.659574


- The months of February through May had more than a 100% trip target achievement rate across all cities
- The month of june had 88% trip target achieved which is the lowest

### New Passenger Target Achievement by Month

In [88]:
# get the actual new passengers per month
month_actual_new_passengers = passenger_summary.groupby(['month_index', 'month_name'], as_index=False).agg(
    new_passengers = ('new_passengers', 'sum')
)

month_actual_new_passengers

,month_index,month_name,new_passengers
0,1,January,36329
1,2,February,36201
2,3,March,30814
3,4,April,26620
4,5,May,24182
5,6,June,22852


In [89]:
# get the month target new passengers
month_new_passengers_target = month_target_new_passenger.groupby(['month_index', 'month_name'], as_index=False).agg(
    target_new_passengers = ('target_new_passengers', 'sum')
)

month_new_passengers_target

,month_index,month_name,target_new_passengers
0,1,January,36700
1,2,February,36700
2,3,March,36700
3,4,April,25000
4,5,May,25000
5,6,June,25000


In [90]:
#merge the two dataframe
actual_target_passengers_month = month_actual_new_passengers.merge(month_new_passengers_target, on=['month_index', 'month_name'])
actual_target_passengers_month

,month_index,month_name,new_passengers,target_new_passengers
0,1,January,36329,36700
1,2,February,36201,36700
2,3,March,30814,36700
3,4,April,26620,25000
4,5,May,24182,25000
5,6,June,22852,25000


In [91]:
#calculate the target achievement rate
actual_target_passengers_month['new_passenger_target_achievement_rate'] = actual_target_passengers_month['new_passengers'] * 100 / actual_target_passengers_month['target_new_passengers']
actual_target_passengers_month

,month_index,month_name,new_passengers,target_new_passengers,new_passenger_target_achievement_rate
0,1,January,36329,36700,98.989101
1,2,February,36201,36700,98.640327
2,3,March,30814,36700,83.961853
3,4,April,26620,25000,106.480000
4,5,May,24182,25000,96.728000
5,6,June,22852,25000,91.408000


- The month of April had more than 100% achievement on the new passengers target, january and february had about 98% new passenger target achievement
- The month of march had about 83% of the target achieved, which is the lowest

### New Passenger Target Achievement rate by City

In [92]:
#getting the actual new passengers by city
city_actual_new_passenger = passenger_summary.groupby(['city_name'], as_index=False).agg(
    actual_new_passengers = ('new_passengers', 'sum')
)

city_actual_new_passenger

,city_name,actual_new_passengers
0,Chandigarh,18908
1,Coimbatore,8514
2,Indore,14863
3,Jaipur,45856
4,Kochi,26416
5,Lucknow,16260
6,Mysore,11681
7,Surat,11626
8,Vadodara,10127
9,Visakhapatnam,12747


In [93]:
#gettting the target new passengers by city 
city_target_new_passenger = month_target_new_passenger.groupby(['city_name'], as_index=False).agg(
    target_new_passengers = ('target_new_passengers', 'sum')
)

city_target_new_passenger

,city_name,target_new_passengers
0,Chandigarh,21000
1,Coimbatore,7500
2,Indore,14100
3,Jaipur,54000
4,Kochi,27000
5,Lucknow,15600
6,Mysore,12000
7,Surat,10500
8,Vadodara,9900
9,Visakhapatnam,13500


In [94]:
#merge the two dataframe
city_actual_target_new_passengers = city_actual_new_passenger.merge(city_target_new_passenger, on='city_name')
city_actual_target_new_passengers

,city_name,actual_new_passengers,target_new_passengers
0,Chandigarh,18908,21000
1,Coimbatore,8514,7500
2,Indore,14863,14100
3,Jaipur,45856,54000
4,Kochi,26416,27000
5,Lucknow,16260,15600
6,Mysore,11681,12000
7,Surat,11626,10500
8,Vadodara,10127,9900
9,Visakhapatnam,12747,13500


In [95]:
#create the target achievement rate 
city_actual_target_new_passengers['target_achievement_rate'] = city_actual_target_new_passengers['actual_new_passengers'] *100/ city_actual_target_new_passengers['target_new_passengers']
city_actual_target_new_passengers

,city_name,actual_new_passengers,target_new_passengers,target_achievement_rate
0,Chandigarh,18908,21000,90.038095
1,Coimbatore,8514,7500,113.520000
2,Indore,14863,14100,105.411348
3,Jaipur,45856,54000,84.918519
4,Kochi,26416,27000,97.837037
5,Lucknow,16260,15600,104.230769
6,Mysore,11681,12000,97.341667
7,Surat,11626,10500,110.723810
8,Vadodara,10127,9900,102.292929
9,Visakhapatnam,12747,13500,94.422222


In [96]:
#display the result in descending order of target achievement rate
city_actual_target_new_passengers.sort_values('target_achievement_rate', ascending=False)

,city_name,actual_new_passengers,target_new_passengers,target_achievement_rate
1,Coimbatore,8514,7500,113.520000
7,Surat,11626,10500,110.723810
2,Indore,14863,14100,105.411348
5,Lucknow,16260,15600,104.230769
8,Vadodara,10127,9900,102.292929
4,Kochi,26416,27000,97.837037
6,Mysore,11681,12000,97.341667
9,Visakhapatnam,12747,13500,94.422222
0,Chandigarh,18908,21000,90.038095
3,Jaipur,45856,54000,84.918519


- Five(5) cities Coimbatore, Surat, Indore, Lucknow, Vadodara were able to achieve more than 100% of their new passenger targets
- Jaipur had the lowest achievement of 84% for their new passenger targets

## Average Passenger Rating Target Achievement

#### For new trips

In [97]:
#get the average passenger ratings by city for new trips
actual_avg_passenger_rating = trips.query("passenger_type=='new'").groupby(['city_name'], as_index=False).agg(
    actual_avg_passenger_rating = ('passenger_rating', 'mean')
)

actual_avg_passenger_rating

,city_name,actual_avg_passenger_rating
0,Chandigarh,8.489158
1,Coimbatore,8.485788
2,Indore,8.485837
3,Jaipur,8.985018
4,Kochi,8.987394
5,Lucknow,7.977429
6,Mysore,8.982964
7,Surat,7.984173
8,Vadodara,7.979263
9,Visakhapatnam,8.976151


#### For Repeated Trips

In [98]:
#get the average passenger ratings by city for repeated trips
actual_repeat_avg_passenger_rating = trips.query("passenger_type=='repeated'").groupby(['city_name'], as_index=False).agg(
    actual_avg_passenger_rating = ('passenger_rating', 'mean')
)

actual_repeat_avg_passenger_rating

,city_name,actual_avg_passenger_rating
0,Chandigarh,7.493798
1,Coimbatore,7.475457
2,Indore,7.473961
3,Jaipur,7.991042
4,Kochi,8.003665
5,Lucknow,5.985741
6,Mysore,7.978495
7,Surat,5.995511
8,Vadodara,5.978629
9,Visakhapatnam,7.989628


In [99]:
#merge the table for new trips rating with the average target ratings
actual_target_avg_passenger_ratings = actual_avg_passenger_rating.merge(ratings, on='city_name').drop('city_id', axis=1)
actual_target_avg_passenger_ratings

,city_name,actual_avg_passenger_rating,target_avg_passenger_rating
0,Chandigarh,8.489158,8.00
1,Coimbatore,8.485788,8.25
2,Indore,8.485837,8.00
3,Jaipur,8.985018,8.25
4,Kochi,8.987394,8.50
5,Lucknow,7.977429,7.25
6,Mysore,8.982964,8.50
7,Surat,7.984173,7.00
8,Vadodara,7.979263,7.50
9,Visakhapatnam,8.976151,8.50


In [100]:
#create the target achievement rate
actual_target_avg_passenger_ratings['target_achievement_rate'] = actual_target_avg_passenger_ratings['actual_avg_passenger_rating'] * 100 / actual_target_avg_passenger_ratings['target_avg_passenger_rating']
actual_target_avg_passenger_ratings.sort_values('target_achievement_rate', ascending=False)

,city_name,actual_avg_passenger_rating,target_avg_passenger_rating,target_achievement_rate
7,Surat,7.984173,7.00,114.059620
5,Lucknow,7.977429,7.25,110.033507
3,Jaipur,8.985018,8.25,108.909313
8,Vadodara,7.979263,7.50,106.390178
0,Chandigarh,8.489158,8.00,106.114475
2,Indore,8.485837,8.00,106.072966
4,Kochi,8.987394,8.50,105.734047
6,Mysore,8.982964,8.50,105.681927
9,Visakhapatnam,8.976151,8.50,105.601779
1,Coimbatore,8.485788,8.25,102.858038


- All cities achieved achieved more than a 100% passenger ratings target for new trips

In [101]:
#merge the table for repeat trips rating with the average target ratings
actual_target_avg_passenger_ratings_repeats = actual_repeat_avg_passenger_rating.merge(ratings, on='city_name').drop('city_id', axis=1)
actual_target_avg_passenger_ratings_repeats

,city_name,actual_avg_passenger_rating,target_avg_passenger_rating
0,Chandigarh,7.493798,8.00
1,Coimbatore,7.475457,8.25
2,Indore,7.473961,8.00
3,Jaipur,7.991042,8.25
4,Kochi,8.003665,8.50
5,Lucknow,5.985741,7.25
6,Mysore,7.978495,8.50
7,Surat,5.995511,7.00
8,Vadodara,5.978629,7.50
9,Visakhapatnam,7.989628,8.50


In [102]:
#create the target achievement rate
actual_target_avg_passenger_ratings_repeats['target_achievement_rate'] = actual_target_avg_passenger_ratings_repeats['actual_avg_passenger_rating'] * 100 / actual_target_avg_passenger_ratings_repeats['target_avg_passenger_rating']
actual_target_avg_passenger_ratings_repeats.sort_values('target_achievement_rate', ascending=False)

,city_name,actual_avg_passenger_rating,target_avg_passenger_rating,target_achievement_rate
3,Jaipur,7.991042,8.25,96.861109
4,Kochi,8.003665,8.50,94.160761
9,Visakhapatnam,7.989628,8.50,93.995624
6,Mysore,7.978495,8.50,93.864643
0,Chandigarh,7.493798,8.00,93.672470
2,Indore,7.473961,8.00,93.424510
1,Coimbatore,7.475457,8.25,90.611597
7,Surat,5.995511,7.00,85.650158
5,Lucknow,5.985741,7.25,82.561941
8,Vadodara,5.978629,7.50,79.715055


- No cities met their target for passenger ratings for repeat trips

### Question 4: Average Fare Per Trip Analysis

In [103]:
avg_calculations = trips.groupby(['city_name'], as_index=False).agg(
    total_trips = ('trip_id', 'count'),
    total_fare = ('fare_amount', 'sum'),
    total_distance_travelled = ('distance_travelled(km)', 'sum'),
    avg_trip_distance = ('distance_travelled(km)', 'mean')
    
)

#calculating avg_fare_per_trip
avg_calculations['avg_fare_per_trip'] = avg_calculations['total_fare'] / avg_calculations['total_trips']
avg_calculations['avg_fare_per_distance'] = avg_calculations['total_fare'] / avg_calculations['total_distance_travelled']

avg_calculations

,city_name,total_trips,total_fare,total_distance_travelled,avg_trip_distance,avg_fare_per_trip,avg_fare_per_distance
0,Chandigarh,38981,11058401,916783,23.518714,283.686950,12.062179
1,Coimbatore,21104,3523992,316121,14.979198,166.982183,11.147605
2,Indore,42456,7635228,700629,16.502473,179.838609,10.897676
3,Jaipur,76888,37207497,2308418,30.023125,483.918128,16.118180
4,Kochi,50702,16997596,1220167,24.065461,335.245079,13.930549
5,Lucknow,64299,9463551,804571,12.512963,147.180376,11.762232
6,Mysore,16238,4054745,267877,16.496921,249.707168,15.136593
7,Surat,54843,6431599,603122,10.997247,117.272925,10.663844
8,Vadodara,32026,3797200,368867,11.517736,118.566165,10.294225
9,Visakhapatnam,28366,8018282,639765,22.553938,282.672284,12.533168


### Trip demands by city and percentage contribution

In [104]:
#group by city and calculate total trips
city_trips_demand = trips.groupby(['city_name'], as_index=False).agg(
    total_trips = ('trip_id', 'count')
).sort_values('total_trips', ascending=False)

#create a new column percentage_contribution
city_trips_demand['%_contribution'] = round((city_trips_demand['total_trips']*100)/trips['trip_id'].count(), 1)

#display the result
city_trips_demand

,city_name,total_trips,%_contribution
3,Jaipur,76888,18.1
5,Lucknow,64299,15.1
7,Surat,54843,12.9
4,Kochi,50702,11.9
2,Indore,42456,10.0
0,Chandigarh,38981,9.2
8,Vadodara,32026,7.5
9,Visakhapatnam,28366,6.7
1,Coimbatore,21104,5.0
6,Mysore,16238,3.8


# APP DEVELOPMENT

In [105]:
#create the app
app = Dash(__name__, external_stylesheets=[dbc.themes.CERULEAN], assets_folder='assets', assets_url_path='/assets/')

load_figure_template("CERULEAN")


#create the layout
app.layout = dbc.Container([
    
    #creating tab
    dbc.Tabs([
        
        #tab 1: Trip Demand Analysis
        dbc.Tab(
            label = "Trip Demand Insights",
            children = [
                
                #empty row
                dbc.Row(
                    html.Br()
                ),
                
                #Row for heading
                dbc.Row([
                    html.Br(),
                    dbc.Col(width = 3),
                    dbc.Col(
                        html.H2("Trip Demand by Month & City")
                    )
                ]),

                #Row 1
                dbc.Row([
                    #column 1
                    dbc.Col([

                        dbc.Card([
                            html.Br(),
                            html.H3(id="trips_total"),
                            html.Span("Total Trips"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="new_trips_total"),
                            html.H4(id="per_new_trips"),
                            html.Span("New Trips"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="repeat_trips_total"),
                            html.H4(id="per_repeat_trips"),
                            html.Span("Repeat Trips"),
                            html.Br()
                        ], style={'text-align': 'center'})
                    ], width=3),

                    #column 2
                    dbc.Col(
                        dbc.Card([
                            
                            html.Span("Select Trip Type:"),
                            dcc.RadioItems(
                                id = "trip_demand_type",
                                options = ['Month', 'City'],
                                value = "Month",
                                inline = True
                            ),
                            dcc.Graph(id="trip_by_month_graph")
                        ])
                    )#closing bracket for column 2
                ]),


                #empty row
                dbc.Row(
                    html.Br()
                ),
                
                #row for heading
                dbc.Row([
                    dbc.Col(width=3),
                    dbc.Col(
                        html.H2("Peak and Low Trip Demand by City")
                    )
                ]),
                
                
                #row for Peak and low demand
                dbc.Row([
                    #column 1: empty
                    dbc.Col(width=3),
                    
                    #column 2
                    dbc.Col([
                        
                        dbc.Card([
                            html.H5("Select City to Analyze Trend: "),

                            dcc.Checklist(
                                id = "select_city1",
                                options = list(trips['city_name'].unique()),
                                value = ['Jaipur', 'Lucknow', 'Chandigarh'],
                                inline=True
                            ),

                            dcc.Graph(id = "peak_low_demand_graph")
                        ])
                    ])
                    
                ]),
                
                
                #empty row
                dbc.Row(
                    html.Br()
                ),
                
                # title row for row 2
                dbc.Row([
                    html.Br(),
                    dbc.Col(width=3),
                    dbc.Col(
                        html.H2("Weekday vs. Weekend Trip Demand"),
                    )
                ]),

                #Row 2
                dbc.Row([
                    #column 1
                    dbc.Col([
                        dbc.Card([
                            html.Br(),
                            html.H3(id="weekday_trips_total"),
                            html.H4(id="per_weekday"),
                            html.Span("Weekday Trips"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="weekend_trips_total"),
                            html.H4(id="per_weekend"),
                            html.Span("Weekend Trips"),
                            html.Br()
                        ], style={'text-align': 'center'})
                    ], width=3),

                    #column 2
                    dbc.Col([
                        
                        dbc.Card([
        
                            dcc.Graph(id = "weektype_graph")
                        ])
                    ])
                ]),


                #empty row
                dbc.Row(
                    html.Br()
                ),
                
                
                dbc.Row([
                    html.Br(),
                    dbc.Col(width=3),
                    dbc.Col(
                        html.H3("New vs. Repeat Trip Ratio"),
                    )
                ]),

                #Row 3
                dbc.Row([
                    #column 1
                    dbc.Col(
                        dbc.Card([
                            html.Br(),
                            html.H3(id="new_repeat_trips_ratio"),
                            html.Span("New-Repeat Trip Ratio"),
                            html.Br()
                        ], style={'text-align': 'center'}), width=3
                    ),

                    #column 2
                    dbc.Col([
                            
                            dbc.Card([
                                
                                html.Span("Select Type:"),
                                dcc.RadioItems(
                                    id="select_new_repeat_ratio_type",
                                    options = ["Month", "City"],
                                    value="Month",
                                    inline=True
                                ),
                                dcc.Graph(id="new_repeat_trip")
                        ])
                    ])
                ])

                        ]
        
        ),#closing tab for tab 1
        
        
        
        #Tab 2: For Passenger Analysis
        dbc.Tab(
            label = "Passengers Insight",
            children = [
                
                dbc.Row([
                dbc.Col(width=3),
                dbc.Col(
                     html.H2("New vs. Repeat Passenger Ratio")
                    )
                ]),

                #row for new passenger repeat ratio
                dbc.Row([

                    #col 1 for metrics
                    dbc.Col([

                        dbc.Card([
                            html.Br(),
                            html.H3(id="total_passengers"),
                            html.Span("Total Passengers"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="new_passengers"),
                            html.H4(id="new_pass_per"),
                            html.Span("New Passengers"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="repeat_passengers"),
                            html.H4(id="repeat_pass_per"),
                            html.Span("Repeat Passengers"),
                            html.Br()
                        ], style={'text-align': 'center'})



                    ], width=3),

                    dbc.Col(
                        dbc.Card([
                            html.Span("select type:"),
                            dcc.RadioItems(
                                id="select_new_repeat_type",
                                options = ["Month", "City"],
                                value="Month",
                                inline=True
                                ),
                            dcc.Graph(id="new_repeat_passenger")
                        ])
                    )

                ]),

                #empty row
                dbc.Row(
                    html.Br()
                ),

                dbc.Row([
                    dbc.Col(width = 3),

                    dbc.Col([
                        html.Br(),
                        html.H2("Repeat Passenger Rate Analysis")
                    ])


                ]),

                #row for repeat passenger rate
                dbc.Row([

                    #empty column
                    dbc.Col(width=3),

                    #col 1
                    dbc.Col(
                        dbc.Card([
                            html.Span("Select type:"),
                            dcc.RadioItems(
                                    id = "rpr_options",
                                    options = ['Month', 'City'],
                                    value = "Month",
                                    inline = True
                                ),
                            dcc.Graph(id="rpr_graph")
                        ])
                    )


                ]),

                dbc.Row([
                    dbc.Col(width=3),
                    #col2
                    dbc.Col([
                        html.Span(id="explain_chart"),
                        html.Br()
                    ]),
                    html.Br()
                ])
                
            ]
        
        ),#closing barcket for tab 2: passenger insight
        
        #Tab 3: Target Achievement Rate
        dbc.Tab(
            label = "Target Achievement Analysis",
            children = [
                
                #row 1: for heading and select type of analysis
                dbc.Row([
                    html.H2("Target Achievement Rate Analysis"),
                    html.Span("Target achievement rate measures how well the goodcabs met their targets "),
                    html.Br(),
                    html.Br(),
                    html.Span("Select Type: "),
                    dcc.RadioItems(
                        id = "target_achievement_options",
                        options = ['Trip Targets', 'New Passenger Targets', 'Passenger Satisfaction Ratings'],
                        value = 'Trip Targets',
                        inline=True
                    )
                ], style={'color': 'white'}),

                #empty row
                dbc.Row(
                    html.Br()
                ),

                #row 2: For displaying two column charts by month and city
                dbc.Row([

                    #col 1 for KPI metrics
                    dbc.Col([
                        dbc.Card([
                            html.Br(),
                            html.H3(id="target_value"),
                            html.Span(id = "target_text"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="actual_value"),
                            html.Span(id="actual_text"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="per_achievement"),
                            html.Span(id = "per_text"),
                            html.Br()
                        ], style={'text-align': 'center'})

                    ]),

                    #col 2 for month target
                    dbc.Col(
                        dbc.Card([
                            dcc.Graph(
                                id ="month_target_graph"
                            )
                        ]), width=5
                    ),

                    #col 3 for city target
                    dbc.Col(
                        dbc.Card([
                            dcc.Graph(
                                id = "city_target_graph"
                            )
                        ]), width=5
                    )
                ])
                
            ]
        
        ), #closing bracket for tab 3: Target achievement analysis
        
        
        #tab 4: Revenue Analysis
        dbc.Tab(
            
            label = "Revenue Insight",
            children = [
                #Row 0 for heading
                dbc.Row([
                    dbc.Col(width=3),
                    dbc.Col(
                        html.H3("Revenue Analysis")
                    )
                ]),

                #Row 1 for revenue analysis by month and city
                dbc.Row([

                    dbc.Col(

                        dbc.Card([
                            html.Br(),
                            html.Br(),
                            html.H3(id="total_revenue_generated"),
                            html.Span("Total Revenue"),
                            html.Br(),
                            html.Br(),
                        ],style={'text-align': 'center'}), width=3
                    ),

                    dbc.Col([

                        dbc.Card([
                            html.Span("Select Type:"),
                            dcc.RadioItems(
                                id = "select_revenue_type",
                                options = ['Month', 'City'],
                                value = "Month",
                                inline=True
                            ),

                            dcc.Graph(id="revenue_graph")
                        ])
                    ])

                ]),

                #empty row
                dbc.Row(
                    [html.Br(),
                    html.Br()]
                ),

                #Row 2 for Revenue Growth Rate
                dbc.Row([

                    #col for explanation of rgr
                    dbc.Col([
                        html.Span()

                    ], width = 3),

                    dbc.Col([
                        html.H3("Revenue Growth Rate Analysis"),
                        dbc.Card([
                            dcc.Graph(id="revenue_growth_rate_graph")
                        ])
                    ])

                ])
            ]
            
        ), #closing bracket for tab 4: revenue analysis
        
        
        #tab 5: Average trip, fare insight
        dbc.Tab(
            
            label = "Pricing Analysis",
            children = [
                #row for title
                dbc.Row([
                    dbc.Col(width=3),
                    dbc.Col(
                        html.H3("Cities Average Price Analysis")
                    )
                ]),

                #row for metrics and chart
                dbc.Row([

                    #col 1 for metrics
                    dbc.Col([
                        dbc.Card([
                            html.Br(),
                            html.H3(id="total_distance_travelled"),
                            html.Span("Total distance (km)"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="avg_trip_distance"),
                            html.Span("Avg. Trip Distance"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="avg_fare_per_trip"),
                            html.Span("Avg. Trip Cost"),
                            html.Br()
                        ], style={'text-align': 'center'}),

                        html.Br(),

                        dbc.Card([
                            html.Br(),
                            html.H3(id="avg_fare_per_distance"),
                            html.Span("Avg Fare Per Distance"),
                            html.Br()
                        ], style={'text-align': 'center'})
                    ], width=3),

                    #Col 2 for chart
                    dbc.Col([
                        dbc.Card([
                            html.Span("Select Type: "),
                            dcc.RadioItems(
                                id="avg_select_options",
                                options = ['Avg_Distance - Trip_cost', "Avg_Distance - Avg_fare_per_distance"],
                                value = "Avg_Distance - Trip_cost"
                            ),

                            dcc.Graph(id="avg_graph")
                        ])
                    ])

                ])
            ]
        
        ), #closing bracket for tab 5: average analysis
        
        
    ]) #this bracket is for closing of parent Tabs()
    
    
],
style={'background-image': 'url("/assets/bg_img.jpg")',
       'background-size': 'auto', 'background-repeat': 'repeat-y',
       'background-position': 'fixed', 'height': '380vh'}

)#this is the closing bracket for the Container()


#create the callback for tab1
@app.callback(
    Output("trips_total", "children"),
    Output("new_trips_total", "children"),
    Output("per_new_trips", "children"),
    Output("repeat_trips_total", "children"),
    Output("per_repeat_trips", "children"),
    Output("trip_by_month_graph", "figure"), #added this
    Output("peak_low_demand_graph", "figure"),
    Output("weekday_trips_total", "children"),
    Output("per_weekday", "children"),
    Output("weekend_trips_total", "children"),
    Output("per_weekend", "children"),
    Output("weektype_graph", "figure"),
    Output("new_repeat_trips_ratio", "children"),
    Output("new_repeat_trip", "figure"),
    Input("trip_demand_type", "value"), #added this
    Input("select_city1", "value"),
    Input("select_new_repeat_ratio_type", "value")
)
def plot_peak_low_demand (trip_demand_type, selected_city, select_new_repeat_ratio_type):
    
    trips_total = trips["trip_id"].count()
    new_trips_total = trips.query("passenger_type=='new'")['trip_id'].count()
    per_new_trips = "("+str(round((new_trips_total *100 )/ trips['trip_id'].count(), 1))+ "%"+")"
    repeat_trips_total = trips.query("passenger_type=='repeated'")['trip_id'].count()
    per_repeat_trips = "("+str(round((repeat_trips_total *100) / trips['trip_id'].count(), 1)) + "%"+")"
    
    
    weekday_trips_total = trips.merge(calendar, on=['date', 'month_name']).query("day_type=='Weekday'")['trip_id'].count()
    per_weekday = "("+str(round((weekday_trips_total * 100) / trips['trip_id'].count(), 1))+"%"+")"
    weekend_trips_total = trips.merge(calendar, on=['date', 'month_name']).query("day_type=='Weekend'")['trip_id'].count()
    
    per_weekend = "("+str(round((weekend_trips_total * 100) / trips['trip_id'].count(), 1))+"%"+")"
    
    
    #addition 
    if f"{trip_demand_type}" == "Month":
        fig0 = px.line(
                trips.groupby(['month_index', 'month_name'], as_index=False).agg(total_trips = ('trip_id', 'count')),
                x = "month_name",
                y = "total_trips",
                labels = {
                    "month_name": "Month",
                    "total_trips": "Total Trips"
                },
                title = "Trip Demands by Month"
            ).update_layout(
                title = {
                    "x":0.5,
                    "y": 0.85
                }
            )
    elif f"{trip_demand_type}" == "City":
        fig0 = px.bar(
                city_trips_demand,
                x = "city_name",
                y = "%_contribution",
                title = "% Share of Trips by Cities",
                labels = {
                    "city_name": "Cities",
                    "%_contribution": "% Trip Contribution"
                }
            ).update_layout(
                title = {
                    "x":0.5,
                    "y": 0.85
                }
            )
    
    
    fig = px.line(
                trips.query(f"city_name == {selected_city}").groupby(['month_index', 'month_name', 'city_name'], as_index=False).agg(total_trips = ('trip_id', 'count')),
                x = "month_name",
                y = "total_trips",
                color = 'city_name',
                labels={
                    "month_name": "Month",
                    "total_trips": "Total Trips"
                },
                title = "Monthly Trip Demand by Cities"
                
            ).update_layout(
                legend=dict(x=0.5, y=-0.2, xanchor='center', yanchor='top'),
                legend_orientation='h',
                title = {
                    "x": 0.5,
                    "y": 0.85
                },
                legend_title ="Cities"
            )
    
    #define the variable weektype
    weektype = trips.merge(calendar, on=['date', 'month_name'])
    week_type_analysis = weektype.groupby(['city_name', 'day_type'], as_index=False).agg(
    total_trips = ('trip_id', 'count')
        )
    
    fig1 = px.bar(
        week_type_analysis,
        x = "city_name",
        y = "total_trips",
        color = "day_type",
        barmode="group",
        labels={
            "city_name": "Cities",
            "total_trips": "Total Trips"
        },
        title="Weekday vs. Weekend Trip Demand by Cities"
        ).update_layout(
                title={
                    "x":0.5,
                    "y": 0.85
                },
                legend_title ="Week Type" 
            )
    
    #selection for 
    new_repeat_trips_ratio = round(trips.query("passenger_type=='new'")['trip_id'].count() / trips.query("passenger_type=='repeated'")['trip_id'].count(), 1)
    
    if f"{select_new_repeat_ratio_type}" == "Month":
        fig3 = px.bar(
            new_repeat_trip_ratio,
            x = "month_name",
            y = "new_repeat_trip_ratio",
            labels={
                "month_name": "Month",
                "new_repeat_trip_ratio":"Trip Ratio"
            },
            title = "New vs. Repeat Trip Ratio by Month"
            ).update_layout(
                title={
                    "x":0.5,
                    "y": 0.85
                }
            )
    elif f"{select_new_repeat_ratio_type}" == "City":
        fig3 = px.bar(
            city_new_repeat_trip_ratio.sort_values('new_repeat_trip_ratio', ascending=True),
            y = "city_name",
            x = "new_repeat_trip_ratio",
            labels={
                "city_name": "Cities",
                "new_repeat_trip_ratio": "Trip Ratio"
            },
            title = "New vs. Repeat Trip Ratio by Cities"
            ).update_layout(
                title={
                    "x":0.5,
                    "y": 0.85
                }
            )
    
    return f"{trips_total:,}", f"{new_trips_total:,}", per_new_trips, f"{repeat_trips_total:,}", per_repeat_trips, fig0, fig, f"{weekday_trips_total:,}", per_weekday, f"{weekend_trips_total:,}", per_weekend, fig1, new_repeat_trips_ratio, fig3



#create callback for tab 2: Passenger Insight
#create the callback
@app.callback(
    Output("total_passengers", "children"),
    Output("new_passengers", "children"),
    Output("new_pass_per", "children"),
    Output("repeat_passengers", "children"),
    Output("repeat_pass_per", "children"),
    Output("new_repeat_passenger", "figure"),
    Input("select_new_repeat_type", "value")
)
def plot_new_repeat_chart(selected_new_repeat_type):
    if f"{selected_new_repeat_type}" == "Month":
        fig = px.bar(
            new_repeat_passenger_month,
            x = "month_name",
            y = "new_repeat_passenger_ratio",
            labels={
                "month_name": "Month",
                "new_repeat_passenger_ratio":"New-Repeat Passenger Ratio"
            },
            title = "New vs. Repeat Passenger Ratio by Month"
            ).update_layout(
                title = {
                "x":0.5,
                "y": 0.85}
            )
    elif f"{selected_new_repeat_type}" == "City":
        fig = px.bar(
            new_repeat_passenger_city.sort_values('new_repeat_passenger_ratio', ascending=True),
            y = "city_name",
            x = "new_repeat_passenger_ratio",
            labels={
                "city_name": "Cities",
                "new_repeat_passenger_ratio":"New-Repeat Passenger Ratio"
            },
            title = "New vs. Repeat Passenger Ratio by City"
        
            ).update_layout(
                title = {
                "x":0.5,
                "y": 0.85}
            )
    
    
    total_passengers = passenger_summary['total_passengers'].sum()
    new_passengers = passenger_summary['new_passengers'].sum()
    new_pass_per = "("+ str(round((new_passengers * 100) / total_passengers, 1))+"%"+ ")"
    
    repeat_passengers = passenger_summary['repeat_passengers'].sum()
    repeat_pass_per = "("+ str(round((repeat_passengers * 100) / total_passengers, 1))+"%"+")"
    
    return f"{total_passengers:,}", f"{new_passengers:,}", new_pass_per, f"{repeat_passengers:,}", repeat_pass_per, fig


# call back for the second row

#create the callback
@app.callback(
    Output("rpr_graph", "figure"),
    Output("explain_chart", "children"),
    Input("rpr_options", "value")
)
def plot_rpr_chart(rpr_option_selected):
    if f"{rpr_option_selected}" == "City":
        explain_chart = f"Surat, Lucknow and Indore are the top three cities with high repeat passenger rates. This implies that in passengers in this three cities are loyal"
        fig = px.bar(
                    rpr_city.sort_values('%_rpr', ascending=False),
                    x = 'city_name',
                    y = '%_rpr',
                    title = 'Repeat passenger rate by city'
                ).update_layout(
                     title = {
                         "x": 0.5,
                         "y": 0.85
                     }
                 ).update_yaxes(
                     title = "% of Repeat Passenger"
                 ).update_xaxes(
                     title = "Cities"
                 )
    elif f"{rpr_option_selected}" == "Month":
        explain_chart = f"There is a steady increase in repeat passenger rate which peaked in May to about 33%"
        fig = px.line(
                    rpr_month,
                    x = "month_name",
                    y = '%_rpr',
                    title = 'Repeat Passenger Rate by Month'
                ).update_layout(
                     title = {
                         "x": 0.5,
                         "y": 0.85
                     }
                 ).update_yaxes(
                     title = "% of Repeat Passenger"
                 ).update_xaxes(
                     title = "Month"
                 )
        
   
        
    return fig, explain_chart


#creating callback for tab 3: target achievement
#create the callback 
@app.callback(
    Output("target_value", "children"),
    Output("target_text", "children"),
    Output("actual_value", "children"),
    Output("actual_text", "children"),
    Output("per_achievement", "children"),
    Output("per_text", "children"),
    Output("month_target_graph", "figure"),
    Output("city_target_graph", "figure"),
    Input("target_achievement_options", "value")
)
def plot_target_achievement_charts(selected_target_option):

    
    if f"{selected_target_option}" == "Trip Targets":
        target_value = monthly_target_trips['total_target_trips'].sum()
        target_text = "Target Trips"
        actual_value = trips['trip_id'].count()
        actual_text = "Actual Trips",
        per_achievement = str(round((actual_value *100)/target_value, 1))+"%"
        per_text = "% Achievement Rate"
        
        fig1 = px.bar(
            month_actual_target_trips,
            x = "month_name",
            y = "target_achievement_rate",
            title = "Trip Target Achievement by Month",
            labels = {
                "month_name": "Month",
                "target_achievement_rate": "% Target Achievement Rate"
                
            }
        ).update_layout(
            title={
            "x": 0.5,
            "y": 0.85}
        ).add_hline(
            y = 100,
            line_dash = "dash",
            line_color = "green"
        )
        
        fig2 = px.bar(
            city_actual_target_trips.sort_values('target_achievement_rate', ascending=True),
            y = "city_name",
            x = "target_achievement_rate",
            title = "Trip Target Achievement by City",
            labels = {
                "city_name": "Cities",
                "target_achievement_rate": "% Target Achievement Rate"
                
            }
        ).update_layout(
            title={
            "x": 0.5,
            "y": 0.85}
        ).add_vline(
            x = 100,
            line_dash = "dash",
            line_color = "green"
        )
    elif f"{selected_target_option}" == "New Passenger Targets":
        
        target_value = month_target_new_passenger['target_new_passengers'].sum()
        target_text = "Target New Passengers"
        actual_value = passenger_summary['new_passengers'].sum()
        actual_text = "Actual New Passengers"
        per_achievement = str(round((actual_value *100)/target_value, 1))+"%"
        per_text = "% Achievement Rate"
        
        fig1 = px.bar(
            actual_target_passengers_month,
            x = "month_name",
            y = "new_passenger_target_achievement_rate",
            title = "New Passenger Target Achievement by Month",
            labels = {
                "month_name": "Month",
                "new_passenger_target_achievement_rate": "% Target Achievement Rate"
                
            }
        ).update_layout(
            title={
            "x": 0.5,
            "y": 0.85}
        ).add_hline(
            y = 100,
            line_dash = "dash",
            line_color = "green"
        )
        
        fig2 = px.bar(
            city_actual_target_new_passengers.sort_values('target_achievement_rate', ascending=True),
            y = "city_name",
            x = "target_achievement_rate",
            title = "New Passenger Target Achievement by City",
            labels = {
                "city_name": "Cities",
                "target_achievement_rate": "% Target Achievement Rate"
                
            }
        ).update_layout(
            title={
            "x": 0.5,
            "y": 0.85}
        ).add_vline(
            x = 100,
            line_dash = "dash",
            line_color = "green"
        )
    elif f"{selected_target_option}" == "Passenger Satisfaction Ratings": #-------------------------------------------------
        
        target_value = round(ratings['target_avg_passenger_rating'].mean(), 1)
        target_text = "Target Passengers Rating"
        actual_value = round(trips.groupby(['city_name'],as_index=False).agg(avg_passenger_rate = ('passenger_rating', 'mean'))['avg_passenger_rate'].mean(), 1)
        actual_text = "Actual Passengers Rating"
        per_achievement = str(round((actual_value *100)/target_value, 1))+"%"
        per_text = "% Achievement Rate"
        
        fig1 = px.bar(
            actual_target_avg_passenger_ratings.sort_values('target_achievement_rate', ascending=True),
            y = "city_name",
            x = "target_achievement_rate",
            title = "New Trips Passenger Ratings",
            labels = {
                "city_name": "Cities",
                "target_achievement_rate": "% Target Achievement Rate"
                
            }
        ).update_layout(
            title={
            "x": 0.5,
            "y": 0.85}
        ).add_vline(
            x = 100,
            line_dash = "dash",
            line_color = "green"
        )
        
        fig2 = px.bar(
            actual_target_avg_passenger_ratings_repeats.sort_values('target_achievement_rate', ascending=True),
            y = "city_name",
            x = "target_achievement_rate",
            title = "Repeated Trip Passenger Ratings",
            labels = {
                "city_name": "Cities",
                "target_achievement_rate": "% Target Achievement Rate"
                
            }
        ).update_layout(
            title={
            "x": 0.5,
            "y": 0.85}
        ).add_vline(
            x = 100,
            line_dash = "dash",
            line_color = "green"
        )      
        
    return f"{target_value:,}", target_text, f"{actual_value:,}", actual_text, per_achievement, per_text, fig1, fig2 



#create callback for tab 4: revenue analysis
#define the callback
@app.callback(
    Output("total_revenue_generated", "children"),
    Output("revenue_graph", "figure"),
    Output("revenue_growth_rate_graph", "figure"),
    Input("select_revenue_type", "value")
)
def plot_revenue (select_revenue_type):
    if f"{select_revenue_type}" == "Month":
        fig = px.line(
            revenue_month,
            x = "month_name",
            y = "current_revenue",
            title = "Revenue trend by month",
            labels = {
                "month_name": "Month",
                "current_revenue": "Revenue"
            }
        ).update_layout(
        title = {
            "x": 0.5,
            "y": 0.85
        }
        )
    elif f"{select_revenue_type}" == "City":
        fig = px.bar(
            revenue_city,
            x = "city_name",
            y = "%_revenue_contribution",
            title = "% revenue contribution by cities",
            labels = {
                "city_name": "City",
                "%_revenue_contribution": "% Revenue Share"
            }
        ).update_layout(
        title = {
            "x": 0.5,
            "y": 0.85
        }
        )
    
    fig2 = px.bar(
        revenue_month,
        x = "month_name",
        y = "revenue_growth_rate",
        title = "Revenue Growth Rate by Month",
        labels = {
            "month_name": "Month",
            "revenue_growth_rate": "Revenue Growth Rate"
        }
    ).update_layout(
        title = {
            "x": 0.5,
            "y": 0.85
        }
    )
    
    total_revenue_generated = trips['fare_amount'].sum()
    
    return f"{total_revenue_generated:,}", fig, fig2


#create callback for the average tab
#create the callback 
@app.callback(
    Output("total_distance_travelled", "children"),
    Output("avg_trip_distance", "children"),
    Output("avg_fare_per_trip", "children"),
    Output("avg_fare_per_distance", "children"),
    Output("avg_graph", "figure"),
    Input("avg_select_options", "value")
    
)
def plot_avg_graph(avg_select_options):
    if f"{avg_select_options}" == "Avg_Distance - Trip_cost":
        fig = px.scatter(
                avg_calculations,
                y = "avg_trip_distance",
                x = "avg_fare_per_trip",
                color="city_name",
                labels = {
                    "avg_trip_distance": "Avg Trip Distance",
                    "avg_fare_per_trip": "Avg Fare per Trip"
                },
                title = "Average Trip Distance vs. Average Fare per Trip"
            ).add_vline(
                       x = sum(trips['fare_amount']) / trips['trip_id'].count(),
                       line_dash = "dash",
                       line_color = "green"
                       ).add_hline(
                        y = trips['distance_travelled(km)'].mean(),
                        line_dash = "dash",
                        line_color = "green"
                        ).update_layout(
                            title = {
                                "x": 0.5,
                                "y": 0.85
                            },
                            legend_title = "Cities"
                        )
    elif f"{avg_select_options}" == "Avg_Distance - Avg_fare_per_distance":
            fig = px.scatter(
                        avg_calculations,
                        y = "avg_trip_distance",
                        x = "avg_fare_per_distance",
                        color="city_name",
                        labels = {
                                    "avg_trip_distance": "Avg Trip Distance",
                                    "avg_fare_per_distance": "Avg Fare per Distance"
                                },
                        title = "Average Trip Distance vs. Average Fare per Distance"
                    ).add_hline(
                               y = trips['distance_travelled(km)'].mean(),
                               line_dash = "dash",
                               line_color = "green"
                               ).add_vline(
                                x = sum(trips['fare_amount']) / sum(trips['distance_travelled(km)']),
                                line_dash = "dash",
                                line_color = "green"
                                ).update_layout(
                                    title = {
                                        "x": 0.5,
                                        "y": 0.85
                                    },
                                    legend_title = "Cities"
                                )
            
    
    total_distance_travelled = trips['distance_travelled(km)'].sum()
    avg_trip_distance = round(trips['distance_travelled(km)'].mean(), 1)
    avg_fare_per_trip = round(trips['fare_amount'].sum()/ trips['trip_id'].count(), 1)
    avg_fare_per_distance = round(trips['fare_amount'].sum()/ trips['distance_travelled(km)'].sum(), 1)
        
    return f"{total_distance_travelled:,}", avg_trip_distance, avg_fare_per_trip, avg_fare_per_distance, fig


#run the app
if __name__ == "__main__":
    app.run_server(port=8025, jupyter_mode="tab")
    


Dash app running on http://127.0.0.1:8025/


<IPython.core.display.Javascript object>

## Business Request 3: CITY-LEVEL REPEAT PASSENGER TRIP FREQUENCY REPORT

In [107]:
trip_freq = repeat_trip.pivot_table(
    columns="trip_count",
    index = "city_name",
    values="repeat_passenger_count",
    aggfunc = "sum"
)

trip_freq = trip_freq.reindex(
    labels = ['2-Trips', '3-Trips', '4-Trips', '5-Trips', '6-Trips', '7-Trips', '8-Trips', '9-Trips', '10-Trips'], axis=1
)

In [108]:
#display the trip_freq
trip_freq

trip_count,2-Trips,3-Trips,4-Trips,5-Trips,6-Trips,7-Trips,8-Trips,9-Trips,10-Trips
city_name,,,,,,,,,
Chandigarh,1638,976,798,619,376,278,176,118,91
Coimbatore,286,378,397,526,450,267,157,59,31
Indore,2478,1637,967,746,494,378,235,172,109
Jaipur,4855,2007,1173,609,400,244,184,116,94
Kochi,3635,1857,901,494,298,161,126,92,62
Lucknow,927,1417,1555,1768,1937,1087,617,183,106
Mysore,720,361,188,86,60,26,21,8,7
Surat,843,1232,1430,1706,1594,1027,539,150,117
Vadodara,429,616,718,785,829,559,251,89,70
